<a href="https://colab.research.google.com/github/hrmartin/hrmartin/blob/main/Colab_TrainNetwork_VideoAnalysis_TF2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepLabCut 2+ Toolbox | Colab for TensorFlow 2+
https://github.com/DeepLabCut/DeepLabCut

![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1607087333173-TG4DRQR5WZ6ETXAJHXEO/ke17ZwdGBToddI8pDm48kKdgWGJOTWy1pqJZDMO95txZw-zPPgdn4jUwVcJE1ZvWQUxwkmyExglNqGp0IvTJZUJFbgE-7XRK3dMEBRBhUpwsA4PS9IFCXiEkiYtvZKsVufHa_vZp4Er3aXh9hA8Hg33vCVX1-CYF3ghyfBsW8js/dlcTF.png?format=1500w)

This notebook illustrates how to use colab:
- create a training set
- train a network
- evaluate a network
- create simple quality check plots
- analyze novel videos, filter data, create videos

###This notebook assumes you already have a project folder with labeled data.

NOTE: This notebook demonstrates the necessary steps to use DeepLabCut for your own project.

This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper.

Nath\*, Mathis\* et al.: Using DeepLabCut for markerless pose estimation during behavior across species. Nature Protocols, 2019.

Paper: https://www.nature.com/articles/s41596-019-0176-0

Pre-print: https://www.biorxiv.org/content/biorxiv/early/2018/11/24/476531.full.pdf


## First, go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU"


In [ ]:
#Install the branch with tf2.x support:
#!pip install git+https://github.com/DeepLabCut/DeepLabCut-core.git@tf2.2alpha
!pip install deeplabcut
!pip install tf_slim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 591 kB 31.5 MB/s 
     |████████████████████████████████| 11.2 MB 63.3 MB/s 
     |████████████████████████████████| 109 kB 67.8 MB/s 
     |████████████████████████████████| 177 kB 70.2 MB/s 
     |████████████████████████████████| 296 kB 87.6 MB/s 
     |████████████████████████████████| 9.8 MB 69.5 MB/s 
     |████████████████████████████████| 948 kB 70.9 MB/s 
     |████████████████████████████████| 352 kB 88.9 MB/s 
     |████████████████████████████████| 3.5 MB 73.4 MB/s 
     |████████████████████████████████| 944 kB 56.4 MB/s 
     |████████████████████████████████| 34.6 MB 1.5 MB/s 
     |████████████████████████████████| 546 kB 66.3 MB/s 
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110474 sha256=d98000fa455490e5d8ee5bff9cd66e726f93ad84bb9075c7cb888a65310b1371
  Stored in directory: /root/.cache/pip/wheels/ce

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**(Be sure to click "RESTART RUNTIME" is it is displayed above above before moving on !)**

In [ ]:
import deeplabcut

Loading DLC 2.2.1.1...
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


In [ ]:
#I don't know if I actually need this cell lol
import tensorflow
tensorflow.__version__

'2.8.2'

## Link your Google Drive:

### First, place your whole porject folder into you google drive! "i.e. move the folder named "Project-YourName-TheDate" into google drive. This should be done after labeling, but before creating a training set. 

In [ ]:
#Now, let's link to your GoogleDrive. Run this cell and follow the authorization instructions:
#(We recommend putting a copy of the github repo in your google drive if you are using the demo "examples")

from google.colab import drive
drive.mount('/content/drive')

# it was unsuccessful to try and mount my hrmartin@uchicago.edu drive while running colab from vestibularhaircell account - HRM

Mounted at /content/drive


YOU WILL NEED TO EDIT THE PROJECT PATH **in the config.yaml file** TO BE SET TO YOUR GOOGLE DRIVE LINK!

You can simply double click on the file on the left, and edit within colab to have the correct path name set.

Typically, this will be: /content/drive/My Drive/yourProjectFolderName


In [ ]:
#Setup your project variables:
# PLEASE EDIT THESE:
ProjectFolderName = 'beam-DS-2022-07-26'
VideoType = 'MOV' 

#don't edit these:
videofile_path = ['/content/drive/My Drive/TF2Projects/'+ProjectFolderName+'/videos/'] #Enter the list of videos or folder to analyze.
print('videofile_path:   ', videofile_path[0])

#This creates a path variable that links to your google drive copy
#No need to edit this, as you set it up before: 
path_config_file = '/content/drive/My Drive/TF2Projects/'+ProjectFolderName+'/config.yaml'
print('path_config_file: ',path_config_file)

videofile_path:    /content/drive/My Drive/TF2Projects/beam-DS-2022-07-26/videos/
path_config_file:  /content/drive/My Drive/TF2Projects/beam-DS-2022-07-26/config.yaml


In [ ]:
#This cell outputs a copy-and-pasteable list of the video file paths in case we need to replace the ones in the config file.
#I don't think we'll need to replace them; in fact, replacing them is not allowed currently.
import glob

a = glob.glob([videofile_path[0]+'*.'+VideoType][0])

for file in a:
  print(' ', file)
  print('   ','crop: 0, 658, 0, 210')


  /content/drive/My Drive/TF2Projects/beam-DS-2022-07-26/videos/Copy of 220715-2547-W3.mov
    crop: 0, 658, 0, 210


## Create a training dataset:
### You must do this step inside of Colab:
After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

Now it is the time to start training the network!

In [ ]:
#Set the shuffle you want to create, train, evaluate, use for analysis, etc (1 is default):
SHUF = 1 

In [ ]:
#ls drive/MyDrive/TF2Projects/DarkMouse-ES-2022-07-06/videos
deeplabcut.__version__

'2.2.1.1'

In [ ]:
from pickle import TRUE
# Note: if you are using the demo data (i.e. examples/Reaching-Mackenzie-2018-08-30/), first delete the folder called dlc-models! 
#Then, run this cell. There are many more functions you can set here, including which network to use.
#check the docstring for full options you can do!
deeplabcut.create_training_dataset(path_config_file, net_type='resnet_50', Shuffles=[SHUF], augmenter_type='imgaug',windows2linux=True)

/usr/local/lib/python3.7/dist-packages/deeplabcut/generate_training_dataset/trainingsetmanipulation.py:823: FutureWarning: `windows2linux` has no effect since 2.2.0.4 and will be removed in 2.2.1.
  FutureWarning,


/content/drive/My Drive/TF2Projects/beam-DS-2022-07-26/labeled-data/220715-2505-N3/CollectedData_DS.h5  not found (perhaps not annotated).
/content/drive/My Drive/TF2Projects/beam-DS-2022-07-26/labeled-data/220715-2539-N1/CollectedData_DS.h5  not found (perhaps not annotated).
/content/drive/My Drive/TF2Projects/beam-DS-2022-07-26/labeled-data/220718-2525-W2/CollectedData_DS.h5  not found (perhaps not annotated).
/content/drive/My Drive/TF2Projects/beam-DS-2022-07-26/labeled-data/220718-2525-W3/CollectedData_DS.h5  not found (perhaps not annotated).
/content/drive/My Drive/TF2Projects/beam-DS-2022-07-26/labeled-data/220718-2526-N2/CollectedData_DS.h5  not found (perhaps not annotated).
/content/drive/My Drive/TF2Projects/beam-DS-2022-07-26/labeled-data/220718-2526-N3/CollectedData_DS.h5  not found (perhaps not annotated).
/content/drive/My Drive/TF2Projects/beam-DS-2022-07-26/labeled-data/220718-2526-W2/CollectedData_DS.h5  not found (perhaps not annotated).
/content/drive/My Drive/TF2

[(0.95,
  1,
  (array([893, 993, 983, ..., 203,  84, 633]),
   array([ 908,  611,  398,   91, 1106,  430,  779, 1143,  291,   57, 1345,
           257,  888, 1529, 1466, 1066,  388,  335,  273,  756, 1077, 1485,
           128,  802,   28, 1207,  882, 1483,  510,  675,  151,  244,  714,
           543,  544,  639,  697, 1289, 1312,  423,  910, 1171,  659,  797,
          1513, 1201,   99,  850,  448, 1455,  755,  976,  115,  777,   72,
           845,  537,  849,  174,   87,  551, 1510,  705,  314, 1420,  600,
          1496, 1094,  599,  277, 1033, 1383,  763,  835, 1216,  559,  684])))]

## Start training:
This function trains the network for a specific shuffle of the training dataset. 

In [ ]:
#let's also change the display and save_iters just in case Colab takes away the GPU... 
#if that happens, you can reload from a saved point. Typically, you want to train to 200,000 + iterations.
#more info and there are more things you can set: https://github.com/DeepLabCut/DeepLabCut/blob/master/docs/functionDetails.md#g-train-the-network

deeplabcut.train_network(path_config_file, shuffle=SHUF, displayiters=500,saveiters=1000,maxiters=300000)

#this will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end (default, 1.03M iterations). 
#Whichever you chose, you will see what looks like an error message, but it's not an error - don't worry....

Config:
{'all_joints': [[0], [1], [2], [3], [4], [5], [6], [7], [8]],
 'all_joints_names': ['Nose',
                      'TipOfBack',
                      'BellyMidpoint',
                      'TailBase',
                      'TailTip',
                      'FrontPaw',
                      'HindPaw',
                      'BeamStart',
                      'BeamEnd'],
 'alpha_r': 0.02,
 'apply_prob': 0.5,
 'batch_size': 1,
 'clahe': True,
 'claheratio': 0.1,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_beamJul26/beam_DS95shuffle1.mat',
 'dataset_type': 'imgaug',
 'decay_steps': 30000,
 'deterministic': False,
 'display_iters': 1000,
 'edge': False,
 'emboss': {'alpha': [0.0, 1.0], 'embossratio': 0.1, 'strength': [0.5, 1.5]},
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'histeq': True,
 'histeqratio': 0.1,
 'init_weights': '/usr/local/lib/python3.7/dist-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50

Selecting single-animal trainer
Batch Size is 1


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tf_slim/layers/layers.py:684: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  outputs = layer.apply(inputs, training=is_training)


Loading ImageNet-pretrained resnet_50
Max_iters overwritten as 300000
Display_iters overwritten as 500
Save_iters overwritten as 1000
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': '/content/drive/My Drive/TF2Projects/beam-DS-2022-07-26/dlc-models/iteration-0/beamJul26-trainset95shuffle1/train/snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'crop_pad': 0, 'scoremap_dir': 'test', 'batch_size': 1, 'dataset_type': 'imgaug', 'deterministic': False, 'mirror': False, 'pairwise_huber_loss': False, 'weigh_only_present_joints': False, 'partaffinityfield_predict': False, 'pairwise_predict': False, 'all_joint

iteration: 500 loss: 0.0306 lr: 0.005
iteration: 1000 loss: 0.0166 lr: 0.005
iteration: 1500 loss: 0.0143 lr: 0.005
iteration: 2000 loss: 0.0130 lr: 0.005
iteration: 2500 loss: 0.0120 lr: 0.005
iteration: 3000 loss: 0.0109 lr: 0.005
iteration: 3500 loss: 0.0106 lr: 0.005
iteration: 4000 loss: 0.0098 lr: 0.005
iteration: 4500 loss: 0.0095 lr: 0.005
iteration: 5000 loss: 0.0087 lr: 0.005
iteration: 5500 loss: 0.0083 lr: 0.005
iteration: 6000 loss: 0.0087 lr: 0.005
iteration: 6500 loss: 0.0083 lr: 0.005
iteration: 7000 loss: 0.0078 lr: 0.005
iteration: 7500 loss: 0.0079 lr: 0.005
iteration: 8000 loss: 0.0076 lr: 0.005
iteration: 8500 loss: 0.0074 lr: 0.005
iteration: 9000 loss: 0.0073 lr: 0.005
iteration: 9500 loss: 0.0073 lr: 0.005
iteration: 10000 loss: 0.0071 lr: 0.005
iteration: 10500 loss: 0.0101 lr: 0.02
iteration: 11000 loss: 0.0095 lr: 0.02
iteration: 11500 loss: 0.0084 lr: 0.02
iteration: 12000 loss: 0.0077 lr: 0.02
iteration: 12500 loss: 0.0077 lr: 0.02
iteration: 13000 loss: 0.

The network is now trained and ready to evaluate. Use the function 'evaluate_network' to evaluate the network.


Exception in thread Thread-13:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/client/session.py", line 1377, in _do_call
    return fn(*args)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/client/session.py", line 1361, in _run_fn
    target_list, run_metadata)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/client/session.py", line 1455, in _call_tf_sessionrun
    run_metadata)
tensorflow.python.framework.errors_impl.CancelledError: Enqueue operation was cancelled
	 [[{{node fifo_queue_enqueue}}]]

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/deeplabcut/pose_estimation_tensorflow/core/train.py", line 83, i

**When you hit "STOP" you will get a KeyInterrupt "error"! No worries! :)**

## Start evaluating:
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [ ]:
#bodypartstocompare = ['FrontRight1','FrontRight2','FrontLeft1','FrontLeft2','HindRight1','HindRight2','HindLeft1','HindLeft2']
#print(bodypartstocompare)

deeplabcut.evaluate_network(path_config_file,Shuffles=[SHUF], plotting=False)#,comparisonbodyparts = bodypartstocompare)

# Here you want to see a low pixel error! Of course, it can only be as good as the labeler, 
#so be sure your labels are good! (And you have trained enough ;). 
# Also note that the test is typically on only a few images, so also good to look
# at video performance (below).

KeyboardInterrupt: ignored

In [ ]:
deeplabcut.evaluate_network?

## There is an optional refinement step you can do outside of Colab:
- if your pixel errors are not low enough, please check out the protocol guide on how to refine your network!
- You will need to adjust the labels **outside of Colab!** We recommend coming back to train and analyze videos... 
- pplease see the repo and protocol instructions on how to refine your data!

## Start Analyzing videos: 
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [ ]:
deeplabcut.analyze_videos(path_config_file, videofile_path, videotype=VideoType, shuffle=SHUF)

Using snapshot-300000 for model /content/drive/My Drive/TF2Projects/beam-DS-2022-07-26/dlc-models/iteration-0/beamJul26-trainset95shuffle1


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tf_slim/layers/layers.py:684: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  outputs = layer.apply(inputs, training=is_training)


Analyzing all the videos in the directory...
Starting to analyze %  /content/drive/My Drive/TF2Projects/beam-DS-2022-07-26/videos/220721_2474_W2.MOV
Loading  /content/drive/My Drive/TF2Projects/beam-DS-2022-07-26/videos/220721_2474_W2.MOV
Duration of video [s]:  12.07 , recorded with  30.0 fps!
Overall # of frames:  362  found with (before cropping) frame dimensions:  1396 262
Starting to extract posture


 99%|█████████▉| 360/362 [00:24<00:00, 14.75it/s]


Saving results in /content/drive/My Drive/TF2Projects/beam-DS-2022-07-26/videos...
Starting to analyze %  /content/drive/My Drive/TF2Projects/beam-DS-2022-07-26/videos/Copy of 220718-2525-W2.MOV
Loading  /content/drive/My Drive/TF2Projects/beam-DS-2022-07-26/videos/Copy of 220718-2525-W2.MOV
Duration of video [s]:  11.72 , recorded with  30.02 fps!
Overall # of frames:  352  found with (before cropping) frame dimensions:  1576 272
Starting to extract posture


...

 99%|█████████▉| 720/729 [00:28<00:00, 25.47it/s]


Saving results in /content/drive/My Drive/TF2Projects/beam-DS-2022-07-26/videos...
Starting to analyze %  /content/drive/My Drive/TF2Projects/beam-DS-2022-07-26/videos/Copy of 220715-2539-N2.MOV
Loading  /content/drive/My Drive/TF2Projects/beam-DS-2022-07-26/videos/Copy of 220715-2539-N2.MOV
Duration of video [s]:  66.57 , recorded with  30.0 fps!
Overall # of frames:  1997  found with (before cropping) frame dimensions:  1624 304
Starting to extract posture


100%|█████████▉| 1995/1997 [01:26<00:00, 22.99it/s]

Saving results in /content/drive/My Drive/TF2Projects/beam-DS-2022-07-26/videos...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.


'DLC_resnet50_beamJul26shuffle1_300000'

You can also now filter your data to smooth any small jitters:

In [ ]:
deeplabcut.filterpredictions(path_config_file, videofile_path, videotype=VideoType, shuffle=SHUF)

Analyzing all the videos in the directory...
Filtering with median model /content/drive/My Drive/TF2Projects/beam-DS-2022-07-26/videos/220721_2471_N2.MOV
Saving filtered csv poses!
Filtering with median model /content/drive/My Drive/TF2Projects/beam-DS-2022-07-26/videos/Copy of 220715-2538-W1.MOV
Saving filtered csv poses!
Filtering with median model /content/drive/My Drive/TF2Projects/beam-DS-2022-07-26/videos/Copy of 220718-2526-W1.MOV
Saving filtered csv poses!
Filtering with median model /content/drive/My Drive/TF2Projects/beam-DS-2022-07-26/videos/Copy of 220718-2523-N3.MOV
Saving filtered csv poses!
Filtering with median model /content/drive/My Drive/TF2Projects/beam-DS-2022-07-26/videos/220721_2471_W1.MOV
Saving filtered csv poses!
Filtering with median model /content/drive/My Drive/TF2Projects/beam-DS-2022-07-26/videos/220721_2474_N2.MOV
Saving filtered csv poses!
Filtering with median model /content/drive/My Drive/TF2Projects/beam-DS-2022-07-26/videos/Copy of 220715-2547-N1.MO

## Plot the trajectories of the analyzed videos:
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
import matplotlib
deeplabcut.plot_trajectories(path_config_file, videofile_path, videotype=VideoType, shuffle=SHUF, filtered=True)

Analyzing all the videos in the directory...
Loading  /content/drive/My Drive/TF2Projects/beam-DS-2022-07-26/videos/Copy of 220715-2547-W3.mov and data.
Plots created! Please check the directory "plot-poses" within the video directory


Now you can look at the plot-poses file and check the "plot-likelihood.png" might want to change the "p-cutoff" in the config.yaml file so that you have only high confidnece points plotted in the video. i.e. ~0.8 or 0.9. The current default is 0.4. 

## Create labeled video:
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [ ]:
deeplabcut.create_labeled_video(path_config_file,videofile_path, videotype=VideoType, shuffle=SHUF, filtered=True)

Analyzing all the videos in the directory...
Starting to process video: /content/drive/My Drive/TF2Projects/beam-DS-2022-07-26/videos/Copy of 220715-2547-W3.mov
Loading /content/drive/My Drive/TF2Projects/beam-DS-2022-07-26/videos/Copy of 220715-2547-W3.mov and data.
Duration of video [s]: 23.59, recorded with 30.01 fps!
Overall # of frames: 708 with cropped frame dimensions: 1386 280
Generating frames and creating video.


100%|██████████| 708/708 [00:03<00:00, 178.90it/s]
